Project 2 : Movie Recommendation System

In [54]:
import os
import plotly.express as px
import numpy as np
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
import re
from sklearn.neighbors import NearestNeighbors
import random
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
!pip install mlxtend

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [55]:
dataNames = pd.read_csv("/content/movies.csv")
dataRatings = pd.read_csv("/content/ratings.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [56]:
dataRatings.head()  # First dataset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,userId,movieId,rating,timestamp
0,1,110,1.0,1.425942e+09
1,1,147,4.5,1.425942e+09
2,1,858,5.0,1.425942e+09
3,1,1221,5.0,1.425942e+09
4,1,1246,5.0,1.425942e+09


In [57]:
dataNames.head()  # Second dataset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [58]:
# Inner join of two datasets (common column is movieId)
data = pd.merge(dataRatings, dataNames, how='inner')
data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,userId,movieId,rating,timestamp,title,genres
0,1,110,1.0,1.425942e+09,Braveheart (1995),Action|Drama|War
1,11,110,3.5,1.231677e+09,Braveheart (1995),Action|Drama|War
2,22,110,5.0,1.111937e+09,Braveheart (1995),Action|Drama|War
3,24,110,5.0,9.798700e+08,Braveheart (1995),Action|Drama|War
4,29,110,3.0,1.044020e+09,Braveheart (1995),Action|Drama|War


In [59]:
# Number of Users who rated at least one movie:
print("Number of Users who rated at least one move: ", data.userId.nunique())
print("-"*25)

# Number of Movies in the dataset:
print("Number of Movies in the dataset:", data.title.nunique())
print("-"*25)

# Unique of Rating points in the dataset:
print("Unique Rating points:", data.rating.unique())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Number of Users who rated at least one move:  1261
-------------------------
Number of Movies in the dataset: 7188
-------------------------
Unique Rating points: [1.  3.5 5.  3.  4.  0.5 4.5 2.  2.5 1.5 nan]


In [60]:
# Extracting movie release years into one column
data['movie_year'] = data.title.str.extract('.*\((.*)\).*')
data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

<>:2: DeprecationWarning:

invalid escape sequence '\('

<>:2: DeprecationWarning:

invalid escape sequence '\('

<ipython-input-60-259254d3619d>:2: DeprecationWarning:

invalid escape sequence '\('



,userId,movieId,rating,timestamp,title,genres,movie_year
0,1,110,1.0,1.425942e+09,Braveheart (1995),Action|Drama|War,1995
1,11,110,3.5,1.231677e+09,Braveheart (1995),Action|Drama|War,1995
2,22,110,5.0,1.111937e+09,Braveheart (1995),Action|Drama|War,1995
3,24,110,5.0,9.798700e+08,Braveheart (1995),Action|Drama|War,1995
4,29,110,3.0,1.044020e+09,Braveheart (1995),Action|Drama|War,1995


In [61]:
# Removing year from the movie titles
data['title'] = data.title.str.split('(').str[0].str[:-1]
data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,userId,movieId,rating,timestamp,title,genres,movie_year
0,1,110,1.0,1.425942e+09,Braveheart,Action|Drama|War,1995
1,11,110,3.5,1.231677e+09,Braveheart,Action|Drama|War,1995
2,22,110,5.0,1.111937e+09,Braveheart,Action|Drama|War,1995
3,24,110,5.0,9.798700e+08,Braveheart,Action|Drama|War,1995
4,29,110,3.0,1.044020e+09,Braveheart,Action|Drama|War,1995


In [62]:
def UNIX_to_Readable(df):
    if pd.notna(df):
        return pd.to_datetime(datetime.fromtimestamp(df).strftime('%Y-%m-%d %H:%M:%S'))
    else:
        return pd.NaT  # Return NaT (Not a Time) for NaN values

# Converting Unix date-format to readable format, handling NaN values
data.timestamp = data.timestamp.apply(UNIX_to_Readable)

# Display the updated DataFrame
print(data.head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



   userId  movieId  rating           timestamp       title            genres  \
0       1      110     1.0 2015-03-09 22:52:09  Braveheart  Action|Drama|War   
1      11      110     3.5 2009-01-11 12:29:49  Braveheart  Action|Drama|War   
2      22      110     5.0 2005-03-27 15:23:29  Braveheart  Action|Drama|War   
3      24      110     5.0 2001-01-19 02:06:52  Braveheart  Action|Drama|War   
4      29      110     3.0 2003-01-31 13:33:25  Braveheart  Action|Drama|War   

  movie_year  
0       1995  
1       1995  
2       1995  
3       1995  
4       1995  


In [63]:
# Removing decimal values to the ceiling value to decrease number of rating classes
data.rating = np.ceil(data.rating)
print("Unique Rating Points:", data.rating.unique())

Unique Rating Points: [ 1.  4.  5.  3.  2. nan]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [64]:
_ = data.title.value_counts()
fig = px.histogram(_, x=_, opacity=0.85, marginal='box',
                   labels={
                       'x': 'Number of Ratings taken'})
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(title_text='Distribution of the Number of Ratings taken by the Movies',
                  title_x=0.5, title_font=dict(size=20))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [65]:
# Removing movies that rated less than 10
movieFrequency_greater_10 = data['movieId'].value_counts()[data['movieId'].value_counts() >= 10].index
data = data[data.movieId.isin(movieFrequency_greater_10)]

print("Minimum Number of Rated Movies after Drop:", data.title.value_counts().nsmallest(5))   # So we achieved to obtain number of ratings taken by users minimum 10

Minimum Number of Rated Movies after Drop: Jiro Dreams of Sushi    10
Bridge to Terabithia    10
Days of Heaven          10
General, The            10
Smokin' Aces            10
Name: title, dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Exploratory Data Analysis

Frequency of the Ratings

In [66]:
rating_val_count = data.rating.value_counts()
fig = px.bar(rating_val_count, x=rating_val_count.index, y=rating_val_count, text=rating_val_count,
             labels={
                 "index": "Ratings",
                 'y': 'Number of Ratings'},
             color=rating_val_count
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Frequency of the Ratings',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Top Frequent the Movie Genres

In [67]:
genres_value_counts = data['genres'].str.split('|', expand=True).stack().value_counts()
fig = px.bar(genres_value_counts, x=genres_value_counts.index, y=genres_value_counts, text=genres_value_counts,
             labels={
                 "index": "Genres",
                 'y': 'Frequency'},
             color=genres_value_counts
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top Frequent the Movie Genres',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Number of Movies for each Genre


In [68]:
unique_movies = data.drop_duplicates('title')
unique_movies = unique_movies['genres'].str.split('|', expand=True).stack().value_counts()
fig = px.bar(unique_movies, x=unique_movies.index, y=unique_movies, text=unique_movies,
             labels={
                 "index": "Genres",
                 'y': 'Number of Movies'},
             color=unique_movies
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Number of Movies for each Genre',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [69]:
data['timeDifferenceAfterRelease'] = data.timestamp.dt.year - data.movie_year.astype(int)
fig = px.histogram(data, x='timeDifferenceAfterRelease', opacity=0.85, marginal='box',
                   labels={
                       'timeDifferenceAfterRelease': 'Time Difference After Release (years)'}
                   )
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(title_text='Year Difference Between Release Date of the Movie and the Rating Date',
                  title_x=0.5, title_font=dict(size=20))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [70]:
num_of_rating_weekly = data.resample('m', on='timestamp').size()
fig = px.line(num_of_rating_weekly, x=num_of_rating_weekly.index, y=num_of_rating_weekly,
              labels={'y': 'Number of Ratings given',
                      'timestamp': 'Months'})
fig.update_layout(title_text='Number of Ratings given by the Users Monthly',
                  title_x=0.5, title_font=dict(size=20))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.update_traces(line=dict(width=3))
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [71]:
unique_title = data.drop_duplicates(['title'])  # Dropping duplicate movie titles
unique_title = unique_title.movie_year.value_counts().reset_index().sort_values('index')  # Count of each years movie frequency
fig = px.line(unique_title, x='index', y='movie_year',
              labels={
                  "index": "Movie Release Year",
                  'movie_year': 'Number of Movies Released'})
fig.update_layout(title_text='Number of Movies Released each Year',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show() #Number of Movies Released each Year¶

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [72]:
unique_title = data.drop_duplicates(['title'])  # Dropping duplicate movie titles
unique_title = unique_title.movie_year.value_counts().head(25)  # Count of each years movie frequency
fig = px.bar(unique_title, x=unique_title.index, y=unique_title, text=unique_title,
             labels={
                 "index": "Movie Release Year",
                 'y': 'Number of Movies Released'},
             color=unique_title
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 25 Years with the most Number of Movies Released',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [73]:
genre_vs_rating = data.groupby(['genres', 'rating']).size().unstack().fillna(0)
year_vs_rating = data.groupby(['movie_year', 'rating']).size().unstack().fillna(0)
movie_vs_rating = data.groupby(['title', 'rating']).size().unstack().fillna(0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [74]:
# Let's calculate the Weighted Average for dataframe rows
def Weighted_Average(df):
    x = []
    for i in range(0, df.shape[0]):
        x.append((np.average(df.iloc[i].index, weights=df.iloc[i].values, axis=0)).round(2))
    return x

# Weighted Average calculation for each movie_vs_rating rows
movie_vs_rating['weightedAverage'] = Weighted_Average(movie_vs_rating)
movie_vs_rating.sort_values('weightedAverage', ascending=False).head()

# Weighted Average calculation for each year_vs_rating rows
year_vs_rating['weightedAverage'] = Weighted_Average(year_vs_rating)

# Weighted Average calculation for each genre_vs_rating rows
genre_vs_rating['weightedAverage'] = Weighted_Average(genre_vs_rating)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [75]:
fig = px.bar(movie_vs_rating, x=movie_vs_rating['weightedAverage'].nlargest(15).index,
             y=movie_vs_rating['weightedAverage'].nlargest(15),
             text=movie_vs_rating['weightedAverage'].nlargest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=movie_vs_rating['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Movie with the highest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [76]:
fig = px.bar(movie_vs_rating, x=movie_vs_rating['weightedAverage'].nsmallest(15).index,
             y=movie_vs_rating['weightedAverage'].nsmallest(15),
             text=movie_vs_rating['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=movie_vs_rating['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Movie with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [77]:
fig = px.bar(year_vs_rating, x=year_vs_rating['weightedAverage'].nlargest(15).index,
             y=year_vs_rating['weightedAverage'].nlargest(15),
             text=year_vs_rating['weightedAverage'].nlargest(15),
             labels={
                 "x": "Years",
                 'y': 'Weighted Rating Averages'},
             color=year_vs_rating['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Years with the highest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [78]:
fig = px.bar(year_vs_rating, x=year_vs_rating['weightedAverage'].nsmallest(15).index,
             y=year_vs_rating['weightedAverage'].nsmallest(15),
             text=year_vs_rating['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Years",
                 'y': 'Weighted Rating Averages'},
             color=year_vs_rating['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Years with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [79]:
fig = px.bar(genre_vs_rating, x=genre_vs_rating['weightedAverage'].nlargest(15).index,
             y=genre_vs_rating['weightedAverage'].nlargest(15),
             text=genre_vs_rating['weightedAverage'].nlargest(15),
             labels={
                 "x": "Genres",
                 'y': 'Weighted Rating Averages'},
             color=genre_vs_rating['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Genres with the highest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [80]:
fig = px.bar(genre_vs_rating, x=genre_vs_rating['weightedAverage'].nsmallest(15).index,
             y=genre_vs_rating['weightedAverage'].nsmallest(15),
             text=genre_vs_rating['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Genres",
                 'y': 'Weighted Rating Averages'},
             color=genre_vs_rating['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Genres with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [81]:
# Creating a new DataFrame for unique movies with their weightedAverages and Genres
_ = data.merge(movie_vs_rating.reset_index()).drop_duplicates('title')[['title', 'genres', 'weightedAverage']]
_.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,title,genres,weightedAverage
0,Braveheart,Action|Drama|War,4.08
319,"Basketball Diaries, The",Drama,3.48
346,"Godfather, The",Crime|Drama,4.55
632,"Godfather: Part II, The",Crime|Drama,4.35
824,Dead Poets Society,Drama,3.95


In [82]:
drama_movies = _[_.genres.str.contains('Drama')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(drama_movies, x=drama_movies['weightedAverage'].nlargest(15).index,
             y=drama_movies['weightedAverage'].nlargest(15),
             text=drama_movies['weightedAverage'].nlargest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=drama_movies['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Drama Movies with the largest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [83]:
drama_movies = _[_.genres.str.contains('Drama')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(drama_movies, x=drama_movies['weightedAverage'].nsmallest(15).index,
             y=drama_movies['weightedAverage'].nsmallest(15),
             text=drama_movies['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=drama_movies['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Drama Movies with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [84]:
action_movies = _[_.genres.str.contains('Action')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(action_movies, x=action_movies['weightedAverage'].nlargest(15).index,
             y=action_movies['weightedAverage'].nlargest(15),
             text=action_movies['weightedAverage'].nlargest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=action_movies['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Action Movies with the largest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [85]:
sci_fi_movies = _[_.genres.str.contains('Sci-Fi')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(sci_fi_movies, x=sci_fi_movies['weightedAverage'].nlargest(15).index,
             y=sci_fi_movies['weightedAverage'].nlargest(15),
             text=sci_fi_movies['weightedAverage'].nlargest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=sci_fi_movies['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Sci-Fi Movies with the largest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [86]:
sci_fi_movies = _[_.genres.str.contains('Sci-Fi')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(sci_fi_movies, x=sci_fi_movies['weightedAverage'].nsmallest(15).index,
             y=sci_fi_movies['weightedAverage'].nsmallest(15),
             text=sci_fi_movies['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=sci_fi_movies['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Sci-Fi Movies with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [87]:
# Created a (movieId: title) dictionary for all movieId's for replacing them with their names
movieId_dict = data.drop_duplicates('title')[['movieId', 'title']].set_index('movieId').to_dict()['title']

# First 5 elements of this dictionary
list(movieId_dict.items())[:5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



[(110, 'Braveheart'),
 (147, 'Basketball Diaries, The'),
 (858, 'Godfather, The'),
 (1221, 'Godfather: Part II, The'),
 (1246, 'Dead Poets Society')]

In [88]:
# Creating a pivot table that has indexes as user ratings, and columns as each movie title
dataRecommendation = data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Replacing dataRecommendation columns with the movie titles
dataRecommendation.columns = dataRecommendation.columns.map(movieId_dict)

dataRecommendation.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



movieId,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Sudden Death,GoldenEye,"American President, The",...,"Big Short, The",Sherlock: The Abominable Bride,Zootopia,Finding Dory,The Nice Guys,Arrival,La La Land,Moana,Rogue One: A Star Wars Story,Logan
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
def encode_units(k):
    if k <= 0:
        return 0
    if k >= 1:
        return 1


sets = dataRecommendation.applymap(encode_units)
sets.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



movieId,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Sudden Death,GoldenEye,"American President, The",...,"Big Short, The",Sherlock: The Abominable Bride,Zootopia,Finding Dory,The Nice Guys,Arrival,La La Land,Moana,Rogue One: A Star Wars Story,Logan
userId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
# Applying Apriori algorithm to matrix that we created before (userId-movieId) and setting min support as 0.15
frequent_itemsets = apriori(sets, min_support=0.15, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.75).sort_values('lift', ascending=False)

rules = rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']]
rules.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



,antecedents,consequents,support,lift,confidence
326,"(Lord of the Rings: The Two Towers, The)","(Lord of the Rings: The Return of the King, Th...",0.157937,4.402655,0.789683
323,"(Lord of the Rings: The Return of the King, Th...","(Lord of the Rings: The Two Towers, The)",0.157937,4.402655,0.880531
325,"(Lord of the Rings: The Return of the King, The)","(Lord of the Rings: The Two Towers, The, Lord ...",0.157937,4.221212,0.753788
324,"(Lord of the Rings: The Two Towers, The, Lord ...","(Lord of the Rings: The Return of the King, The)",0.157937,4.221212,0.884444
327,(Lord of the Rings: The Fellowship of the Ring...,"(Lord of the Rings: The Return of the King, Th...",0.157937,4.064780,0.667785


In [91]:
# I will use NearestNeighbors algorithm that I learnt from the scikit-learn documentation here
knn = NearestNeighbors(n_neighbors=11, metric='cosine', algorithm='brute', n_jobs=-1)
knn.fit(dataRecommendation.values.T)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=11)

In [92]:
# Here is our recommendations for Blade Runner, there will be 7 movie recommendations
recommendation_result = list(knn.kneighbors([dataRecommendation['Blade Runner'].values], 8))

recommendation_result  # As you can see from the results, we obtained cosine angles in the first array, and the second array gives us the dataRecommendation column order, I need to convert it to more readable form

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



[array([[4.44089210e-16, 3.33580350e-01, 3.68135009e-01, 4.00341238e-01,
         4.12349633e-01, 4.26622898e-01, 4.44611227e-01, 4.54438522e-01]]),
 array([[ 293,  535,  556,  410,  522,  518,  499, 1195]])]

In [93]:
# Here is our recommendations for Blade Runner, there will be 7 movie recommendations
recommendation_result = list(knn.kneighbors([dataRecommendation['Blade Runner'].values], 8))

recommendation_result  # As you can see from the results, we obtained cosine angles in the first array, and the second array gives us the dataRecommendation column order, I need to convert it to more readable form

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



[array([[4.44089210e-16, 3.33580350e-01, 3.68135009e-01, 4.00341238e-01,
         4.12349633e-01, 4.26622898e-01, 4.44611227e-01, 4.54438522e-01]]),
 array([[ 293,  535,  556,  410,  522,  518,  499, 1195]])]

Some Movie Recommendations

In [94]:
# Movie Recommendation as Function

def movie_recommendation(movie_name, num_of_recommendations):
    a = dataRecommendation.columns.to_frame().reset_index(drop=True).to_dict()['movieId']
    recommendation_result = list(knn.kneighbors([dataRecommendation[movie_name].values], num_of_recommendations + 1))
    recommendation_result = pd.DataFrame(np.vstack((recommendation_result[1], recommendation_result[0])),
                                         index=['movieId', 'Cosine_Similarity (degree)']).T
    recommendation_result = recommendation_result.drop([0]).reset_index(drop=True)
    recommendation_result.movieId = recommendation_result.movieId.map(a)
    return recommendation_result

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [95]:
movie_recommendation('Amazing Spider-Man, The', 7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,movieId,Cosine_Similarity (degree)
0,The Amazing Spider-Man 2,0.310523
1,Man of Steel,0.360764
2,"Hobbit: An Unexpected Journey, The",0.449705
3,The Hunger Games,0.454165
4,Monsters University,0.457608
5,Iron Man 3,0.466182
6,Snow White and the Huntsman,0.468426


In [96]:
movie_recommendation('Mulan', 7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,movieId,Cosine_Similarity (degree)
0,Tarzan,0.519301
1,Hercules,0.520731
2,"Little Mermaid, The",0.523788
3,Lilo & Stitch,0.567737
4,"Lion King, The",0.574256
5,Antz,0.576001
6,Anastasia,0.594888


In [97]:
movie_recommendation('Final Destination', 7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,movieId,Cosine_Similarity (degree)
0,Final Destination 2,0.440599
1,Thirteen Ghosts,0.612737
2,Idle Hands,0.629770
3,"Mummy Returns, The",0.632376
4,Daredevil,0.641111
5,"Ring, The",0.648687
6,Flightplan,0.650179


In [98]:
movie_recommendation('Shining, The', 7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,movieId,Cosine_Similarity (degree)
0,Alien,0.423125
1,Reservoir Dogs,0.455971
2,"Terminator, The",0.462907
3,"Exorcist, The",0.467678
4,Full Metal Jacket,0.468171
5,Aliens,0.478081
6,Blade Runner,0.484760


In [99]:
movie_recommendation('Pulp Fiction', 7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,movieId,Cosine_Similarity (degree)
0,"Silence of the Lambs, The",0.310351
1,Seven,0.335991
2,"Shawshank Redemption, The",0.340836
3,"Usual Suspects, The",0.356510
4,Forrest Gump,0.378918
5,Terminator 2: Judgment Day,0.405694
6,Reservoir Dogs,0.406394


In [100]:
movie_recommendation("Reservoir Dogs", 7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,movieId,Cosine_Similarity (degree)
0,Pulp Fiction,0.406394
1,Goodfellas,0.409492
2,Full Metal Jacket,0.412320
3,Seven,0.415593
4,"Clockwork Orange, A",0.437140
5,Apocalypse Now,0.448295
6,"Shining, The",0.455971


In [101]:
movie_recommendation('Batman Returns', 7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,movieId,Cosine_Similarity (degree)
0,Highlander,0.549826
1,Conan the Barbarian,0.561207
2,RoboCop,0.563193
3,Beavis and Butt-Head Do America,0.574601
4,Total Recall,0.581637
5,Face/Off,0.581832
6,"Crow, The",0.588396


In [102]:
movie_recommendation('Logan', 7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,movieId,Cosine_Similarity (degree)
0,Guardians of the Galaxy 2,0.437535
1,Arrival,0.466502
2,Doctor Strange,0.505677
3,Rogue One: A Star Wars Story,0.525410
4,Suicide Squad,0.534104
5,Batman v Superman: Dawn of Justice,0.565808
6,Moana,0.574682


In [103]:
movie_recommendation('Matrix, The', 7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,movieId,Cosine_Similarity (degree)
0,"Lord of the Rings: The Fellowship of the Ring,...",0.350972
1,Fight Club,0.352954
2,"Sixth Sense, The",0.356484
3,"Lord of the Rings: The Return of the King, The",0.361181
4,Star Wars: Episode IV - A New Hope,0.376947
5,"Lord of the Rings: The Two Towers, The",0.383626
6,Men in Black,0.385658


In [104]:
movie_recommendation('Casablanca', 7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,movieId,Cosine_Similarity (degree)
0,Rear Window,0.487408
1,Citizen Kane,0.506390
2,"Graduate, The",0.509307
3,North by Northwest,0.520302
4,"Godfather, The",0.525184
5,Annie Hall,0.528853
6,It's a Wonderful Life,0.535228


In [105]:
movie_recommendation("Harry Potter and the Sorcerer's Stone", 7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,movieId,Cosine_Similarity (degree)
0,Harry Potter and the Chamber of Secrets,0.218011
1,Harry Potter and the Prisoner of Azkaban,0.301029
2,Harry Potter and the Goblet of Fire,0.336859
3,Pirates of the Caribbean: The Curse of the Bla...,0.397898
4,Spider-Man,0.417324
5,Harry Potter and the Order of the Phoenix,0.418955
6,Shrek,0.429810


In [106]:
movie_recommendation('Rain Man', 7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,movieId,Cosine_Similarity (degree)
0,Good Will Hunting,0.505300
1,American Beauty,0.517203
2,One Flew Over the Cuckoo's Nest,0.520237
3,As Good as It Gets,0.524625
4,Dead Poets Society,0.526971
5,"Sixth Sense, The",0.527772
6,When Harry Met Sally...,0.529400
